In [1]:
from spiketimes.alignment import align_to
from utils import db_setup_core
from sqlalchemy import select, and_, cast, Integer
from pprint import pprint
import pandas as pd
from queries import select_spike_times, select_ifr, select_neurons, select_recording_sessions, select_waveforms, select_analog_signal_data

In [2]:
engine, metadata = db_setup_core()

In [3]:
neurons = metadata.tables["neurons"]
recordings = metadata.tables["recording_sessions"]

In [4]:
df1 = select_spike_times(engine, metadata, 
                         block_name="opto_1hz", 
                         session_names=("opto_03",), 
                         exp_names=("OPTO",),
                         t_before=10*60,
                         as_df=True, 
                         align_to_block=True)
df1.head(3)

,neuron_id,spike_time_samples,session_name,experiment_name,group_name
0,273,-17991953,opto_03,OPTO,SERT-CHR2
1,273,-17950999,opto_03,OPTO,SERT-CHR2
2,273,-17946893,opto_03,OPTO,SERT-CHR2


In [5]:
df2 = select_ifr(engine, 
                 metadata, 
                 block_name="pre", 
                 session_names=("hamilton_10", 
                                     "hamilton_03", "hamilton_04"),
                 t_after=2*60,
                 group_names=("acute_cit",),
                 align_to_block=True)
df2.head(3)

,neuron_id,ifr,timepoint_s,session_name,group_name
0,1011,0.8202596911,1.0000000000,hamilton_10,acute_cit
1,1011,0.8202850885,2.0000000000,hamilton_10,acute_cit
2,1011,0.8203145798,3.0000000000,hamilton_10,acute_cit


In [6]:
df3 = select_neurons(engine, metadata, group_names=("acute_cit", "acute_sal", "shock", "sham"), exclude_mua=False)
df3.head()

,id,recording_session_id,cluster_id,channel,is_single_unit,session_name,group_name,experiment_name
0,48,5,7,4,1,ESHOCK_06_LOC1,sham,ESHOCK
1,49,5,12,8,1,ESHOCK_06_LOC1,sham,ESHOCK
2,50,5,13,10,1,ESHOCK_06_LOC1,sham,ESHOCK
3,51,5,15,12,1,ESHOCK_06_LOC1,sham,ESHOCK
4,52,5,16,14,1,ESHOCK_06_LOC1,sham,ESHOCK


In [7]:
df4 = select_waveforms(engine, metadata, session_names=("hamilton_07",), units="samples")
df4.head()

,neuron_id,waveform_value,session_name
0,1,-14.520,hamilton_07
1,1,-13.085,hamilton_07
2,1,-11.730,hamilton_07
3,1,-9.675,hamilton_07
4,1,-7.275,hamilton_07


In [3]:
df5 = select_recording_sessions(engine, metadata, exp_names=("HAMILTON",))
df5.head()

,id,session_name,session_date,start_time,group_id,mouse_id,chan_map_id,excluded,group_name,experiment_name
0,22,hamilton_10,2019-05-23,12:53:08,6,None,1,None,acute_cit,HAMILTON
1,23,hamilton_03,2019-05-24,11:01:33,7,None,1,None,acute_sal,HAMILTON
2,24,hamilton_04,2019-05-29,10:45:29,6,None,1,None,acute_cit,HAMILTON
3,25,hamilton_09,2019-05-30,12:04:35,6,None,1,None,acute_cit,HAMILTON
4,26,hamilton_01,2019-06-05,10:19:26,9,None,1,None,chronic_saline,HAMILTON


In [8]:
df6 = select_analog_signal_data(engine, metadata, session_names=("hamilton_07",), block_name="base_shock", as_df=False)

SELECT analog_data.voltage, recording_sessions.session_name, analog_signals.signal_name, analog_data.timepoint_s 
FROM analog_data JOIN session_analog_signals ON session_analog_signals.id = analog_data.signal_id JOIN analog_signals ON analog_signals.id = session_analog_signals.signal_id JOIN recording_sessions ON recording_sessions.id = session_analog_signals.recording_session_id JOIN recording_session_block_times ON recording_sessions.id = recording_session_block_times.recording_session_id JOIN experimental_groups ON experimental_groups.id = recording_sessions.group_id JOIN experiments ON experiments.id = experimental_groups.experiment_id JOIN (SELECT recording_session_config.recording_session_id AS recording_session_id, CAST(recording_session_config.config_value AS INTEGER) AS sampling_rate 
FROM recording_session_config 
WHERE recording_session_config.config = :config_1) AS sampling_rate ON sampling_rate.recording_session_id = recording_sessions.id LEFT OUTER JOIN (SELECT recording_

In [10]:
print(df6.fetchone())

None
